In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from scipy import stats as st
from census import Census
from config import api_key
c = Census(api_key, year=2017)
#!pip install census
Medicare_file = "medicare_charges.csv"

In [ ]:
census_data = c.acs5.get(("NAME", "B19013_001E", 
                          "B01003_001E", 
                          "B01002_001E",
                          "B19301_001E",
                          "B17001_002E",
                          "B23025_005E",
                         "B25077_001E",
                         "B15003_022E"), {'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_005E": "Unemployment Count",
                                      "B25077_001E": "Median Home Value",
                                      "B15003_022E": "Bachelor Count",
                                      "NAME": "Name", "zip code tabulation area": "Zip Code"})
Census_Cleaned = census_pd[["Population",
                            "Median Age",
                            "Per Capita Income",
                            "Zip Code"]]
Census_Cleaned = Census_Cleaned.dropna()
Census_Cleaned.head(10)

In [ ]:
print(len(Census_Cleaned["Zip Code"]))


In [ ]:
Medicare_Cost_df = pd.read_csv(Medicare_file)
Medicare_Cost_Cleaned = Medicare_Cost_df[["Provider Zip Code",
                                          "Provider State", 
                                          "DRG Definition",
                                          "Total Discharges",
                                          "Average Total Payments"]]
Medicare_Cost_Cleaned = Medicare_Cost_Cleaned.rename(columns = {"Provider Zip Code":"Zip Code",
                                                               "Provider State":"State",
                                                               "DRG Definition":"Procedure",
                                                               "Average Total Payments": "Cost"})
Medicare_Cost_df.head()
#print(len(Medicare_Cost_df))

In [ ]:
Medicare_Cost_Cleaned = Medicare_Cost_Cleaned[Medicare_Cost_Cleaned.State == "CA"]
Medicare_Cost_871 = Medicare_Cost_Cleaned[Medicare_Cost_Cleaned.Procedure == "871 - SEPTICEMIA OR SEVERE SEPSIS W/O MV >96 HOURS W MCC"]
Medicare_Cost_871 = Medicare_Cost_871.groupby("Zip Code").mean()
Medicare_Cost_871["Zip Code"] = Medicare_Cost_871.index
#Medicare_Cost_871reset_index(drop = True , inplace = True)


Medicare_Cost_Total = Medicare_Cost_871.rename(columns = {"Cost":"Total Avg 871"})

#Medicare_Cost_Total["Total Avg 871] = Medicare_Cost_Cleaned[Medicare_Cost_Cleaned.Procedure == "871 - SEPTICEMIA OR SEVERE SEPSIS W/O MV >96 HOURS W MCC"].groupby("Zip Code").mean()["Cost"]
Medicare_Cost_Total["Total Avg 291"] = round(Medicare_Cost_Cleaned[Medicare_Cost_Cleaned.Procedure == "291 - HEART FAILURE & SHOCK W MCC"].groupby("Zip Code").mean()["Cost"], 0)
Medicare_Cost_Total["Total Avg 872"] = round(Medicare_Cost_Cleaned[Medicare_Cost_Cleaned.Procedure == "872 - SEPTICEMIA OR SEVERE SEPSIS W/O MV >96 HOURS W/O MCC"].groupby("Zip Code").mean()["Cost"], 0)
Medicare_Cost_Total["Total Avg 470"] = round(Medicare_Cost_Cleaned[Medicare_Cost_Cleaned.Procedure == "470 - MAJOR JOINT REPLACEMENT OR REATTACHMENT OF LOWER EXTREMITY W/O MCC"].groupby("Zip Code").mean()["Cost"], 0)
Medicare_Cost_Total["Total Avg 392"] = round(Medicare_Cost_Cleaned[Medicare_Cost_Cleaned.Procedure == "392 - ESOPHAGITIS, GASTROENT & MISC DIGEST DISORDERS W/O MCC"].groupby("Zip Code").mean()["Cost"], 0)

Medicare_Cost_Total["Total Avg 871"] = round(Medicare_Cost_Total["Total Avg 871"], 0)

Medicare_Cost_Total = Medicare_Cost_Total.dropna()
Medicare_Cost_Total.reset_index(drop = True , inplace = True)
Medicare_Cost_Total


In [ ]:
top_5 = Medicare_Cost_Cleaned["Procedure"].value_counts()[:5].index.tolist()
counts = []
for x in top_5:
    count = Medicare_Cost_Cleaned.loc[Medicare_Cost_Cleaned["Procedure"]==x]["Total Discharges"].count()
    counts.append(count)
    
count_df = pd.DataFrame({"Procedure": top_5, "Count": counts},index=['871', '291', '872', '470', '392']).head()
count_df

In [ ]:
# bar chart showing top 5 most common procedures
count_plot = count_df.plot(kind="bar", rot=0, title='Top 5 Most Frequent Diagnoses')
plt.xlabel("Procedures")
plt.ylabel("Count of Hospitals")
plt.ylim(0, max(counts)+50)
count_plot
plt.savefig('Top 5 Diagnoses')

In [ ]:
Census_Cleaned["Zip Code"] = Census_Cleaned["Zip Code"].astype(int)
Medicare_Cost_Total["Zip Code"] = Medicare_Cost_Total["Zip Code"].astype(int)
merged_df = pd.merge(Medicare_Cost_Total, Census_Cleaned, how="inner", on="Zip Code")
merged_df = merged_df.drop([8])
merged_df.head()

In [ ]:
def boxplot(data):
    data_quartiles = data.quantile([.25,.5,.75])
    data_lowerq = data_quartiles[0.25]
    data_upperq = data_quartiles[0.75]
    data_iqr = data_upperq-data_lowerq
    data_lower_bound = data_lowerq - (1.5*data_iqr)
    data_upper_bound = data_upperq + (1.5*data_iqr)

In [ ]:
age = merged_df["Median Age"]
boxplot(age)
plt.boxplot(age)
plt.show()
plt.savefig('Median Age Boxplot with Outliers.png')

In [ ]:
pop = merged_df["Population"]
boxplot(pop)
plt.boxplot(pop)
plt.show()
plt.savefig('Population Boxplot with Outliers.png')

In [ ]:
income = merged_df["Per Capita Income"]
boxplot(income)
plt.boxplot(income)
plt.show()
plt.savefig('Per Capita Income Boxplot with Outliers.png')

In [ ]:
drg_871 = merged_df["Total Avg 871"]
drg_291 = merged_df["Total Avg 291"]
drg_872 = merged_df["Total Avg 872"]
drg_470 = merged_df["Total Avg 470"]
drg_392 = merged_df["Total Avg 392"]

drg_871_quartiles = drg_871.quantile([.25,.5,.75])
drg_871_lowerq = drg_871_quartiles[0.25]
drg_871_upperq = drg_871_quartiles[0.75]
drg_871_iqr = drg_871_upperq-drg_871_lowerq
drg_871_lower_bound = drg_871_lowerq - (1.5*drg_871_iqr)
drg_871_upper_bound = drg_871_upperq + (1.5*drg_871_iqr)

drg_291_quartiles = drg_291.quantile([.25,.5,.75])
drg_291_lowerq = drg_291_quartiles[0.25]
drg_291_upperq = drg_291_quartiles[0.75]
drg_291_iqr = drg_291_upperq-drg_291_lowerq
drg_291_lower_bound = drg_291_lowerq - (1.5*drg_291_iqr)
drg_291_upper_bound = drg_291_upperq + (1.5*drg_291_iqr)

drg_872_quartiles = drg_872.quantile([.25,.5,.75])
drg_872_lowerq = drg_872_quartiles[0.25]
drg_872_upperq = drg_872_quartiles[0.75]
drg_872_iqr = drg_872_upperq-drg_872_lowerq
drg_872_lower_bound = drg_872_lowerq - (1.5*drg_872_iqr)
drg_872_upper_bound = drg_872_upperq + (1.5*drg_872_iqr)

drg_470_quartiles = drg_470.quantile([.25,.5,.75])
drg_470_lowerq = drg_470_quartiles[0.25]
drg_470_upperq = drg_470_quartiles[0.75]
drg_470_iqr = drg_470_upperq-drg_470_lowerq
drg_470_lower_bound = drg_470_lowerq - (1.5*drg_470_iqr)
drg_470_upper_bound = drg_470_upperq + (1.5*drg_470_iqr)

drg_392_quartiles = drg_392.quantile([.25,.5,.75])
drg_392_lowerq = drg_392_quartiles[0.25]
drg_392_upperq = drg_392_quartiles[0.75]
drg_392_iqr = drg_392_upperq-drg_392_lowerq
drg_392_lower_bound = drg_392_lowerq - (1.5*drg_392_iqr)
drg_392_upper_bound = drg_392_upperq + (1.5*drg_392_iqr)

plot_data = [drg_871, drg_291, drg_872, drg_470, drg_392]
fig = plt.figure(1, figsize=(9, 6))
ax = fig.add_subplot(111)
ax.boxplot(plot_data)
ax.set_xticklabels(['DRG 871', 'DRG 291', 'DRG 872', 'DRG 470'])
plt.ylabel("Average Cost")
plt.show()
plt.savefig('Top 5 DRG Boxplot with Outliers.png')

In [ ]:
out_871 = merged_df.loc[(merged_df["Total Avg 871"]<drg_871_upper_bound)]
out_291 = out_871.loc[(out_871["Total Avg 291"]<drg_291_upper_bound)]
out_872 = out_291.loc[(out_291["Total Avg 872"]<drg_872_upper_bound)]
out_470 = out_872.loc[(out_872["Total Avg 470"]<drg_470_upper_bound)]
Data = out_470.loc[(out_470["Total Avg 392"]<drg_392_upper_bound)]
Data

In [ ]:
def boxplot_clean(x):
    x_quartiles = x.quantile([.25,.5,.75])
    x_lowerq = x_quartiles[0.25]
    x_upperq = x_quartiles[0.75]
    x_iqr = x_upperq-x_lowerq
    x_lower_bound = x_lowerq - (1.5*x_iqr)
    x_upper_bound = x_upperq + (1.5*x_iqr)
    
clean_871 = Data["Total Avg 871"]
clean_291 = Data["Total Avg 291"]
clean_872 = Data["Total Avg 872"]
clean_470 = Data["Total Avg 470"]
clean_392 = Data["Total Avg 392"]

boxplot_clean(clean_871)
boxplot_clean(clean_291)
boxplot_clean(clean_872)
boxplot_clean(clean_470)
boxplot_clean(clean_392)

In [ ]:
plot_data = [clean_871, clean_291, clean_872, clean_470, clean_392]
fig = plt.figure(1, figsize=(9, 6))
ax = fig.add_subplot(111)
ax.boxplot(plot_data)
ax.set_xticklabels(['DRG 871', 'DRG 291', 'DRG 872', 'DRG 470', 'DRG 392'])
plt.ylabel("Average Cost")
plt.show()
plt.savefig('Top 5 DRG Boxplot.png')

In [ ]:
Data["Total"] = Data["Total Avg 871"] + Data["Total Avg 291"] + Data["Total Avg 872"] + Data["Total Avg 470"] + Data["Total Avg 392"]

In [ ]:
def Scatter (x, y):
    slope, intercept, r, p, std_err = st.linregress(Data[x], Data[y])
    plt.scatter(Data[x], Data[y], marker="o", color="blue", alpha = 0.25)
    plt.xlabel(x)
    plt.title(f"{x} compared to Average Hospital costs")
    plt.ylabel("Average Cost")
    PCI_Line = slope* Data[x] + intercept
    plt.plot(Data[x], PCI_Line, "r-")
    
    
    
    total_y = Data.iloc[:,1]
    total_x = Data.iloc[:,8]
    corr = st.pearsonr(total_x, total_y)
    #plt.annotate(f"The r value is {round(corr[0], 2)}", ,fontsize=15,color="b")
    #plt.annotate(f"The p value is {round(corr[1], 2)}",loc = "best",fontsize=15,color="b")
    
    plt.show
Scatter_Age_871 = Scatter("Median Age", "Total Avg 871")
plt.savefig("Scatter_Age_871.PNG")

In [ ]:
st.pearsonr?

In [ ]:
Scatter_PCI = Scatter("Per Capita Income")
plt.savefig("Scatter_PCI.PNG")

In [ ]:
Scatter_Pop = Scatter("Population")
plt.savefig("Scatter_Pop.PNG")

In [ ]:
slope

In [ ]:
###Scatter & Linear Regress lines for Data
###Box Plot for Data
###Presentation
###Correlation Coeffecients
###
###

In [ ]:
total_871proc = Data.iloc[:,1]
median_age = Data.iloc[:,8]
corr_871proc_age = st.pearsonr(median_age, total_871proc)
print(round(corr_871proc_age[0],2))

In [ ]:
total_291proc = Data.iloc[:,3]
median_age = Data.iloc[:,8]
corr_291proc_age = st.pearsonr(median_age, total_291proc)
print(round(corr_291proc_age[0],2))

In [ ]:
total_872proc = Data.iloc[:,4]
median_age = Data.iloc[:,8]
corr_872proc_age = st.pearsonr(median_age, total_872proc)
print(round(corr_872proc_age[0],2))

In [ ]:
total_470proc = Data.iloc[:,5]
median_age = Data.iloc[:,8]
corr_470proc_age = st.pearsonr(median_age, total_470proc)
print(round(corr_470proc_age[0],2))

In [ ]:
total_392proc = Data.iloc[:,6]
median_age = Data.iloc[:,8]
corr_392proc_age = st.pearsonr(median_age, total_392proc)
print(round(corr_392proc_age[0],2))

In [ ]:
population = Data.iloc[:,7]
corr_871proc_pop = st.pearsonr(population, total_871proc)
print(round(corr_871proc_pop[0],2))

In [ ]:
population = Data.iloc[:,7]
corr_291proc_pop = st.pearsonr(population, total_291proc)
print(round(corr_291proc_pop[0],2))

In [ ]:
population = Data.iloc[:,7]
corr_872proc_pop = st.pearsonr(population, total_872proc)
print(round(corr_872proc_pop[0],2))

In [ ]:
population = Data.iloc[:,7]
corr_470proc_pop = st.pearsonr(population, total_470proc)
print(round(corr_470proc_pop[0],2))

In [ ]:
population = Data.iloc[:,7]
corr_392proc_pop = st.pearsonr(population, total_392proc)
print(round(corr_392proc_pop[0],2))

In [ ]:
capita_income = Data.iloc[:,9]
corr_871proc_income = st.pearsonr(capita_income, total_871proc)
print(round(corr_871proc_income[0],2))

In [ ]:
capita_income = Data.iloc[:,9]
corr_291proc_income = st.pearsonr(capita_income, total_291proc)
print(round(corr_291proc_income[0],2))

In [ ]:
capita_income = Data.iloc[:,9]
corr_872proc_income = st.pearsonr(capita_income, total_872proc)
print(round(corr_872proc_income[0],2))

In [ ]:
capita_income = Data.iloc[:,9]
corr_470proc_income = st.pearsonr(capita_income, total_470proc)
print(round(corr_470proc_income[0],2))

In [ ]:
st.ttest_ind(total_871proc, median_age)

In [ ]:
st.ttest_ind(total_871proc, population)

In [ ]:
st.ttest_ind(total_871proc, capita_income)

In [ ]:
st.ttest_ind(total_291proc, median_age)

In [ ]:
st.ttest_ind(total_291proc, population)

In [ ]:
st.ttest_ind(total_291proc, capita_income)

In [ ]:
st.ttest_ind(total_872proc, median_age)

In [ ]:
st.ttest_ind(total_872proc, population)

In [ ]:
st.ttest_ind(total_872proc, capita_income)

In [ ]:
st.ttest_ind(total_470proc, median_age)

In [ ]:
st.ttest_ind(total_470proc, population)

In [ ]:
st.ttest_ind(total_470proc, capita_income)